### **Importing the required packages**

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

### **Reading and Exploring the Data**

In [2]:
data = pd.read_csv('nba_final.csv')

In [3]:
data.head() #print the top 5 rows of the dataframe

,Rk,Player.x,Player_ID,Pos1,Pos2,Age,Tm,G,GS,MP,...,Conference,Role,Fvot,FRank,Pvot,PRank,Mvot,MRank,Score,Play
0,170,A.J. Hammons,hammoaj01,C,NaN,24,DAL,22,0,7.4,...,West,Front,786,123,NaN,NaN,NaN,NaN,83.5,No
1,58,Aaron Brooks,brookaa01,PG,NaN,32,IND,65,0,13.8,...,Est,Back,2474,64,NaN,NaN,NaN,NaN,48.2,No
2,157,Aaron Gordon,gordoaa01,SF,NaN,21,ORL,80,72,28.7,...,Est,Front,22774,29,NaN,NaN,NaN,NaN,40.0,No
3,352,Adreian Payne,paynead01,PF,NaN,25,MIN,18,0,7.5,...,West,Front,861,120,1.0,52.0,NaN,NaN,75.5,No
4,10,Al-Farouq Aminu,aminual01,PF,NaN,26,POR,61,25,29.1,...,West,Front,4971,69,7.0,23.0,NaN,NaN,42.8,No


In [4]:
data.shape  #print the number of rows and columns in the dataframe

(1408, 45)

In [5]:
data.dtypes #print the datatype of the values in each column

,0
Rk,int64
Player.x,object
Player_ID,object
Pos1,object
Pos2,object
Age,int64
Tm,object
G,int64
GS,int64
MP,float64


##### We check the datatypes to verify whether each column has the correct datatype or not and also to know whether there are any object columns which we need to encode.

In [6]:
data.isnull().sum(axis = 0) #print the total number of missing values column-wise

,0
Rk,0
Player.x,0
Player_ID,0
Pos1,0
Pos2,1396
Age,0
Tm,0
G,0
GS,0
MP,0


In [7]:
data.isnull().sum(axis = 1) #print the total number of missing values row-wise

,0
0,6
1,5
2,5
3,3
4,3
...,...
1403,1
1404,1
1405,4
1406,1


In [8]:
data.isnull().sum(axis = 1) > 4   #mark the rows as True if they fulfill the mentioned condition and false otherwise

,0
0,True
1,True
2,True
3,False
4,False
...,...
1403,False
1404,False
1405,False
1406,False


In [9]:
data[data.isnull().sum(axis = 1) > 5] #print the rows which have more than 5 missing values

,Rk,Player.x,Player_ID,Pos1,Pos2,Age,Tm,G,GS,MP,...,Conference,Role,Fvot,FRank,Pvot,PRank,Mvot,MRank,Score,Play
0,170,A.J. Hammons,hammoaj01,C,NaN,24,DAL,22,0,7.4,...,West,Front,786,123,NaN,NaN,NaN,NaN,83.5,No
16,443,Anderson Varejão,varejan01,C,NaN,34,GSW,14,1,6.6,...,West,Front,1724,40,NaN,NaN,NaN,NaN,42.0,No
27,341,Arinze Onuaku,onuakar01,C,NaN,29,ORL,8,0,3.5,...,Est,Front,216,129,NaN,NaN,NaN,NaN,90.0,No
37,60,Bobby Brown,brownbo02,PG,NaN,32,HOU,25,0,4.9,...,West,Back,206,70,NaN,NaN,NaN,NaN,52.0,No
48,378,Brian Roberts,roberbr01,PG,NaN,31,CHO,41,2,10.1,...,Est,Back,484,91,NaN,NaN,NaN,NaN,61.8,No
51,67,Bruno Caboclo,cabocbr01,SF,NaN,21,TOR,9,0,4.4,...,Est,Front,5185,66,NaN,NaN,NaN,NaN,58.5,No
54,302,C.J. Miles,milescj01,SF,NaN,29,IND,76,29,23.4,...,Est,Front,1965,95,NaN,NaN,NaN,NaN,73.0,No
55,455,C.J. Watson,watsocj01,PG,NaN,32,ORL,62,9,16.3,...,Est,Back,369,92,NaN,NaN,NaN,NaN,62.2,No
56,463,C.J. Wilcox,wilcocj01,SG,NaN,26,ORL,22,0,4.9,...,Est,Back,287,93,NaN,NaN,NaN,NaN,62.8,No
62,109,Cheick Diallo,diallch01,PF,NaN,20,NOP,17,0,11.7,...,West,Front,877,119,NaN,NaN,NaN,NaN,81.5,No


In [10]:
data = data[data.isnull().sum(axis = 1) <= 5]  #filter out the rows which has more than 5 missing values

In [11]:
data.isnull().sum()

,0
Rk,0
Player.x,0
Player_ID,0
Pos1,0
Pos2,1359
Age,0
Tm,0
G,0
GS,0
MP,0


In [12]:
data.shape  #check the shape again after row deletion

(1371, 45)

In [13]:
data.drop(columns = 'Pos2', inplace = True) #drop the Pos2 column since more than 95% of its values are missing

In [14]:
data = data.fillna(0) #fill the rest of missing values in the dataframe with 0

In [15]:
data.duplicated().sum() #check for the number of duplicate rows

np.int64(0)

In [16]:
data.drop_duplicates(inplace = True)  #drop the duplicate rows, if any

In [17]:
data.drop(columns = ['Player.x', 'Player_ID'], inplace = True)  #drop these 2 columns since they are not useful for ML process

#### **Encode the categorical columns**

In [18]:
obj_cols = data.select_dtypes('object').columns #select the column names of the columns which are of the object type

In [19]:
obj_cols  #list of columns which are object type

Index(['Pos1', 'Tm', 'Season', 'Conference', 'Role', 'Play'], dtype='object')

In [20]:
le = LabelEncoder()

for col in obj_cols:
  data[col] = le.fit_transform(data[col])

In [21]:
#check for data imbalance
data['Play'].value_counts()

,count
Play,
0,1298
1,73


### **Machine Learning Process**

In [22]:
X = data.drop(columns = 'Play') #store the input columns in X
y = data['Play']  #store the output column in y

In [23]:
#split the data into training and test sets and use stratified sampling since the data is imbalanced
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100, stratify = y)

#### **Mean Centering/Scaling/Standardization of the Data**

- Always perform this process after splitting the data into training and test.

In [24]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### **Apply the Logistic Regression on the data**

In [25]:
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

LogisticRegression()

In [26]:
log_reg.coef_   #print the coefficients (beta1 to beta42), which are weightage of each column

array([[ 0.55599738,  0.20563577,  0.68021607,  0.10722513,  1.57491189,
         1.07973303, -0.67160553,  0.41627396,  0.04806734,  0.02483851,
         0.58406804,  0.34452455, -0.55838499,  0.10080902, -0.18136329,
        -0.64537331,  0.20904553, -0.19200101,  0.148108  , -0.47414991,
        -0.10029033,  0.2153693 ,  0.23042837,  0.75398891,  0.06944723,
         0.19404955, -0.7295217 , -0.71186912,  0.32398553,  0.28350647,
         0.36893066,  0.05812181,  0.17763094,  0.44381383, -0.20531763,
        -0.87151468,  0.81503037, -0.39114073,  0.88706089,  0.01481096,
        -1.18597038]])

In [27]:
y_pred = log_reg.predict(X_test_scaled)

In [28]:
accuracy_score(y_test, y_pred)  #accuracy_score is unreliable in the case of imbalance data, so we don't trust it

0.96

In [29]:
roc_auc_score(y_test, y_pred) #this is the correct performance of the model

np.float64(0.7275641025641026)

#### **To check for the overfitting**

In [30]:
y_pred_train = log_reg.predict(X_train_scaled)

In [31]:
roc_auc_score(y_train, y_pred_train)

np.float64(0.947312470932164)

### **Now we will transform the data with PCA and check the performance of the model using the PCA transformed data**

In [38]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 0.95)

In [39]:
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [40]:
X_train_pca.shape

(1096, 21)

### **Apply the logistic Regression on the pca_transformed data**

In [41]:
log_reg2 = LogisticRegression()
log_reg2.fit(X_train_pca, y_train)

LogisticRegression()

In [42]:
y_pred2 = log_reg2.predict(X_test_pca)

In [43]:
roc_auc_score(y_test, y_pred2)

np.float64(0.7647435897435897)

#### **Check for the overfitting again**

In [44]:
y_pred_train2 = log_reg2.predict(X_train_pca)

roc_auc_score(y_train, y_pred_train2)

np.float64(0.9386917812769916)

### **Apply the Linear Discriminant Analysis(LDA) on the data**

In [45]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()

In [46]:
X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

In [47]:
log_reg3 = LogisticRegression()
log_reg3.fit(X_train_lda, y_train)

LogisticRegression()

In [48]:
y_pred3 = log_reg3.predict(X_test_lda)

In [49]:
roc_auc_score(y_test, y_pred3)

np.float64(0.6942307692307692)

In [50]:
X_train_lda.shape

(1096, 1)

In [ ]:
min(n_classes - 1, features)